# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [3]:
import pandas as pd
import numpy as np
from scipy import stats

In [4]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [6]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

In [7]:
pd.options.display.max_columns = 65 
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,occupbroad,workinschool,email,computerskills,specialskills,firstname,sex,race,h,l,call,city,kind,adid,fracblack,fracwhite,lmedhhinc,fracdropout,fraccolp,linc,col,expminreq,schoolreq,eoe,parent_sales,parent_emp,branch_sales,branch_emp,fed,fracblack_empzip,fracwhite_empzip,lmedhhinc_empzip,fracdropout_empzip,fraccolp_empzip,linc_empzip,manager,supervisor,secretary,offsupport,salesrep,retailsales,req,expreq,comreq,educreq,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,1,0,0,1,0,Allison,f,w,0.0,1.0,0.0,c,a,384.0,0.989360,0.005500,9.527484,0.274151,0.037662,8.706325,1.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,6,1,1,1,0,Kristen,f,w,1.0,0.0,0.0,c,a,384.0,0.080736,0.888374,10.408828,0.233687,0.087285,9.532859,0.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,1,1,0,1,0,Lakisha,f,b,0.0,1.0,0.0,c,a,384.0,0.104301,0.837370,10.466754,0.101335,0.591695,10.540329,1.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,5,0,1,1,1,Latonya,f,b,1.0,0.0,0.0,c,a,384.0,0.336165,0.637370,10.431908,0.108848,0.406576,10.412141,0.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,5,1,1,1,0,Carrie,f,w,1.0,0.0,0.0,c,a,385.0,0.397595,0.180196,9.876219,0.312873,0.030847,8.728264,0.0,some,,1.0,9.4,143.0,9.4,143.0,0.0,0.204764,0.727046,10.619399,0.070493,0.369903,10.007352,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [17]:
w = data[data.race=='w']
print("The number of white-sounding resumes is", len(w))

# number of callbacks for white-sounding names
print("The number of callbacks for white-sounding names is", sum(data[data.race=='w'].call))

# Rate of White Callback
w_rate = sum(data[data.race=='w'].call) / len(w)
print ("The rate of white callback is", w_rate)

The number of white-sounding resumes is 2435
The number of callbacks for white-sounding names is 235.0
The rate of white callback is 0.09650924024640657


In [18]:
b = data[data.race=='b']
print("The number of black-sounding resumes is", len(b))

# number of callbacks for black-sounding names
print("The number of callbacks for black-sounding names is", sum(data[data.race=='b'].call))

# Rate of Black Callback
b_rate = sum(data[data.race=='b'].call) / len(b)
print ("The rate of black callback is", b_rate)

The number of black-sounding resumes is 2435
The number of callbacks for black-sounding names is 157.0
The rate of black callback is 0.06447638603696099


In [10]:
# Check for duplicate observations:
data.duplicated().sum()

0

In [11]:
# Check for missing values in race and call columns.
data[['race','call']].isnull().sum()

race    0
call    0
dtype: int64

<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

## 1. What test is appropriate for this problem? Does CLT apply?

A two-proportion z-test is appropriate for this problem. We can apply central limit theorem (CLT) if there are at least 30 observations in your sample.

## 2. What are the null and alternate hypotheses?

**Null Hypothesis:** The proportion of callbacks for white-sounding names and the proportion of callbacks for black-sounding names are equal.

**Alternative Hypothesis:** The proportion of callbacks for white-sounding names and the proportion of callbacks for black-sounding names are not equal.

## 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.


#### a. Bootstrapping Approach

In [12]:
def diff_of_proportions(data_1,data_2):
    return abs(np.sum(data_1) - np.sum(data_2))

In [22]:
def bs_split_data(data, funct, size=1):
    
    bs_replicates = np.empty(size)
    
    for i in range(size):
        # Permutate the concatenated array
        permutated_data= np.random.permutation(data)
        
        # Split the permutated array into two
        half_count = int(len(data)/2)
        perm_sample_1 = permutated_data[:half_count]
        perm_sample_2 = permutated_data[half_count:]
        
        # Call function
        bs_replicates[i] = func(perm_sample_1,)

In [23]:
# Define bootstrap function that randomly split data
def bs_split_data(data, func, size=1):
    
    # Initialize replicates
    bs_replicates = np.empty(size)
    
    for i in range(size):
        # Permutate the concatenated array: permutated_data
        permuted_data = np.random.permutation(data)
        
        # Split the permuted array into two: perm_sample_1, perm_sample_2
        # Assume data can be split into even numbers
        half_count = int(len(data)/2)
        perm_sample_1 = permuted_data[:half_count]
        perm_sample_2 = permuted_data[half_count:]
        
        # Call function
        bs_replicates[i] = func(perm_sample_1/half_count, perm_sample_2/half_count)
        
    return bs_replicates

In [24]:
# Defining variables
b_call_mean = b.call.mean()
w_call_mean = w.call.mean()

# Acquire 10000 bootstrap replicates using difference of proprotion
bs_replicates = bs_split_data(data.call, diff_of_proportions, size=10000)

bs_p_value = np.sum(bs_replicates>=diff_of_proportions(w_call_mean, b_call_mean))/len(bs_replicates)

print("p-value =", bs_p_value)

p-value = 0.0


In [26]:
calls=np.sum(w.call[w.call==1])

diff_mean = w_call_mean - b_call_mean

all_callbacks = np.array([True] * int(calls) + [False] * int(len(data)-calls))

size = 10000

bs_reps_diff = np.empty(size)

for i in range(size):
    w_bs_replicates = np.sum(np.random.choice(all_callbacks, size=len(w)))
    b_bs_replicates = np.sum(np.random.choice(all_callbacks, size=len(b)))
    
    bs_reps_diff[i] = (w_bs_replicates - b_bs_replicates)/len(b)
    
bs_p_value = np.sum(bs_reps_diff >= diff_mean) / len(bs_reps_diff)

bs_ci = np.percentile(bs_reps_diff, [2.5, 97.5])
bs_mean_diff = np.mean(bs_reps_diff)

print('p-value:', bs_p_value)
print('95% conf. int.:', bs_ci)

p-value: 0.0
95% conf. int.: [-0.01191992  0.01190965]


Conclusion:
p_value is less than 0.05. Null Hypothesis will be rejected. We can conclude that race has a significant impact on the rate of callbacks for resumes.

#### b. Frequentist Statical Approach

#### z_score

In [34]:
w_call = np.sum(data[data.race=='w'].call)
b_call = np.sum(data[data.race=='b'].call)          
phat = (w_call + b_call) / (len(w) + len(b))
z = (diff_mean) / np.sqrt(phat * (1 - phat) * (( 1/ len(w)) + (1/len(b))))
print ('z score:',z)

z score: 4.108412235238472


#### p_value

In [35]:
p = stats.norm.cdf(-z)*2
print ('p-value:',p)

p-value: 3.983885409504418e-05


#### Margin of Error

In [38]:
moe = 1.96 * np.sqrt(phat * (1 - phat) * (( 1/ len(w)) + (1/len(b))))
print('Margin of Error:', moe)

Margin of Error: 0.015281912310894095


#### Confidence Interval

In [40]:
ci = diff_mean + np.array([-1, 1]) * moe
print('Confidence interval:', ci)

Confidence interval: [0.01675094 0.04731477]


**Conclusion:**
The p-value is very low than 5%, so the null hypothesis is rejected. There is sufficient evidence that there is ifference in the proportion of callbacks for white-sounding names and the proportion of callbacks for black-sounding names.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

## 4. Write a story describing the statistical significance in the context or the original problem.

We performed a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes. That analysis tested the effect of race on callback success, and the statistics test indicates that racial discrimination in the United States labor market is significant. Given our sample data and our findings in this project, we can conclude that a black- or white-sounding name on a resume has a significant effect on callbacks.

Based on the result of p-value which is too close to 0. This means that the effect that we see (a difference in the proportion of callbacks between white and balck sounding names) is significant.There is a 95% chance that the true difference of white-sounding call back rates and black-sounding call back rates is between 0.016 and 0.04. This means we are 95% confident that there exists a difference between the two races in which individuals with white-sounding names are favored and called back more often.

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

According to the resuts above, we conclude that a black- or white-sounding name on a resume has a significant effect on callbacks.This analysis does not mean the race/name is the most important factor in callback success because there are more other factors may affect employers' decision, like 'year of experience', 'education', 'resume quality',' applicants' address' or 'different types of jobs', etc.